# Mask R-CNN

- 객체 탐지 모델으로부터 시작

  - 예측된 경계 상자가 정확하면 분할 네트워크가 할 일은 단순

  - 해당 패치에서 어느 픽셀이 캡처된 클래스에 속하는지, 어느 픽셀이 배경의 일부이고 어느 픽셀이 다른 클래스에 속하는지 분류

- 사전 훈련된 탐지 네트워크와 뒤에 사전 훈련된 분할 네트워크를 하나로 이어서 End-to-End 방식으로 훈련하여 전체 파이프라인의 성능이 좋아짐을 보임

- Facebook AI Research(FAIR)

- 주로 Faster R-CNN에 기반

  - 참고
    - https://curt-park.github.io/2017-03-17/faster-rcnn/

    - https://tensorflow.blog/2017/06/05/from-r-cnn-to-mask-r-cnn/
- 3번째 병렬 분기를 추가해 확장함으로써 각 영역의 요소에 대한 이진 마스크 출력

  - 해당 분류기를 통해 분류와 분할을 분리할 수 있음

  - 분할 분기는 N개의 이진 마스크(다른 일반적인 의미론적 분할 모델처럼 클래스당 한 개)를 출력하도록 정의되지만,  
    다른 분기에 의해 예측된 클래스에 대응하는 마스크만 최종 예측과 훈련 손실을 계산할 때 사용됨

  <img src="https://miro.medium.com/max/6060/1*M_ZhHp8OXzWxEsfWu2e5EA.png">

  <sub>[이미지 출처] https://medium.com/@jonathan_hui/image-segmentation-with-mask-r-cnn-ebe6d793272</sub>



## Mask R-CNN Implement

- [런타임 유형] - GPU 설정

- 코드 참고
  - https://www.tensorflow.org/tutorials/images/segmentation
  - https://github.com/tensorflow/examples.git

In [1]:
!pip install -q git+https://github.com/tensorflow/examples.git
!pip install -q -U tfds-nightly

In [2]:
import tensorflow as tf
from tensorflow_examples.models.pix2pix import pix2pix

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from IPython.display import clear_output
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-white')

c:\Users\kksoo\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Oxford-IIIT Pets 데이터 세트를 다운로드

- Parkhi *et al*이 만든 [Oxford-IIIT Pet Dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/) 데이터 세트는 영상, 해당 레이블과 픽셀 단위의 마스크로 구성
- 마스크는 기본적으로 각 픽셀의 레이블
- 각 픽셀은 다음 세 가지 범주 중 하나

  *   class 1 : 애완동물이 속한 픽셀
  *   class 2 : 애완동물과 인접한 픽셀
  *   class 3 : 위에 속하지 않는 경우/주변 픽셀

- 이미지를 뒤집는 간단한 확장을 수행합니다. 또한, 영상이 [0,1]로 정규화

- 분할 마스크에서 1을 빼서 레이블이 {0, 1, 2}이 되도록 함

- 데이터 세트에는 이미 필요한 몫의 시험과 훈련이 포함되어 있으므로 동일한 분할을 계속 사용


### 모델 정의

- 사용된 모델은 수정된 U-Net
- U-Net은 인코더(다운샘플러)와 디코더(업샘플러)를 포함
- 강력한 기능을 학습하고 훈련 가능한 매개변수의 수를 줄이기 위해 미리 훈련된 모델을 인코더로 사용할 수 있음  
- 인코더는 미리 훈련된 MobileNetV2 모델이 될 것이며 이 모델의 중간 출력이 사용
- 디코더는 Pix2pix 튜토리얼의 TensorFlow 예제에서 이미 구현된 업샘플 블록이 될 것
- 3개의 채널을 출력하는 이유는 픽셀당 3개의 가능한 라벨이 있기 때문
- 각 화소가 세 개의 class로 분류되는 다중 분류라고 생각

- 사전 훈련된 MobileNetV2 모델

- 인코더는 전체 학습 과정 중 학습 되지 않음

### 모델 학습

- 손실 함수는 `loss.sparse_categorical_crossentropy`
  
  - 이 손실 함수를 사용하는 이유는 네트워크가 멀티 클래스 예측과 마찬가지로 픽셀마다 레이블을 할당하려고 하기 때문  
  
  - 실제 분할 마스크에서 각 픽셀은 {0,1,2}를 가지고 있고, 이곳의 네트워크는 세 개의 채널을 출력
  
  - 기본적으로 각 채널은 클래스를 예측하는 방법을 배우려고 하고 있으며, `loss.sparse_categical_crossentropy`는 그러한 시나리오에 권장되는 손실
  
  - 네트워크의 출력을 사용하여 픽셀에 할당된 레이블은 가장 높은 값을 가진 채널임  
    이것이 create_mask 함수가 하는 일입니다.



- 모델을 시험해보고 훈련 전에 예측한 것이 무엇인지 알아봄

### 콜백 함수 정의

### 학습된 모델을 활용하여 예측